In [11]:
import os
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import MinMaxScaler

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Define LSTM-based Model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Take the last time step
        return out

# Dataset Class
class PharmacyDataset(Dataset):
    def __init__(self, data, features, target):
        self.data = data
        self.features = features
        self.target = target
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        x = torch.tensor(self.data[self.features].iloc[idx].values, dtype=torch.float32).unsqueeze(0)  # Add sequence dim
        y = torch.tensor(self.data[self.target].iloc[idx], dtype=torch.float32)
        return x, y

# Load and Preprocess Data
def load_data(file_path, scaler=None):
    data = pd.read_csv(file_path)
    
    # Feature Engineering: Focus on individual medication costs
    data['Medication_Cost'] = data['Cost_Per_Medication'] * data['Quantity']
    
    features = ['Age', 'Dosage', 'Quantity', 'Medication_Cost']  # Per-medication features
    target = 'Medication_Cost'  # Predict the cost per medication
    
    # Normalize data
    if not scaler:
        scaler = MinMaxScaler()
        data[features] = scaler.fit_transform(data[features])
    else:
        data[features] = scaler.transform(data[features])
    
    return data, features, target, scaler

# Train Model
def train_model(model, dataloader, criterion, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        for x_batch, y_batch in dataloader:
            optimizer.zero_grad()
            outputs = model(x_batch)
            loss = criterion(outputs.squeeze(), y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(dataloader):.4f}")

# Predict with Model (at the medication level)
def predict_model(model, dataloader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for x_batch, _ in dataloader:
            preds = model(x_batch).squeeze(-1)  # Ensure the last dimension is removed
            predictions.extend(preds.cpu().numpy())  # Convert to numpy and extend the list
    return predictions

# Save the trained model
def save_model(model, model_path):
    torch.save(model.state_dict(), model_path)
    print(f"Model saved to {model_path}")

# Load the trained model
def load_model(model, model_path):
    model.load_state_dict(torch.load(model_path))
    model.eval()
    print(f"Model loaded from {model_path}")

# Step 1: Gather all unique medication names
def gather_medication_names(data_paths):
    all_medications = set()
    for file_path in data_paths:
        data = pd.read_csv(file_path)
        all_medications.update(data['Medication_Name'].unique())
    return list(all_medications)

# Step 2: Train pharmacy models
def train_pharmacy_models(data_paths, output_path, scaler=None):
    pharmacy_models = []
    for file_path in data_paths:
        data, features, target, scaler = load_data(file_path, scaler)
        dataset_obj = PharmacyDataset(data, features, target)
        dataloader = DataLoader(dataset_obj, batch_size=32, shuffle=True)
        
        # Model Initialization
        input_size = len(features)
        model = LSTMModel(input_size=input_size, hidden_size=64, num_layers=2, output_size=1)
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        
        # Train Model
        print(f"Training model for {file_path}...")
        train_model(model, dataloader, criterion, optimizer, epochs=2)
        
        # Save the trained model
        model_path = os.path.join(output_path, f"{os.path.basename(file_path).split('.')[0]}_model.pth")
        save_model(model, model_path)
        
        pharmacy_models.append(model)
    return pharmacy_models

# Step 3: Aggregate pharmacy models into one city-level model
# Step 3: Aggregate pharmacy models into one city-level model
def aggregate_pharmacy_models(pharmacy_models, data_paths, output_path):
    # Define mapping of zones to cities (assuming zones 1, 2, and 3 correspond to city 1, city 2, city 3)
    city_zones_mapping = {1: ['Ph01_Z01_C01', 'Ph02_Z01_C01', 'Ph03_Z01_C01'],
                          2: ['Ph01_Z02_C02', 'Ph02_Z02_C02', 'Ph03_Z02_C02'],
                          3: ['Ph01_Z03_C03', 'Ph02_Z03_C03', 'Ph03_Z03_C03']}
    
    # Initialize dictionary to store city predictions
    city_predictions = {f"City{city_id}": [] for city_id in range(1, 4)}  # Create dict for city-level predictions
    
    # Group pharmacy models by city (zones in each city)
    for city_id, zones in city_zones_mapping.items():
        city_data = []
        
        # Process each zone in the city
        for zone in zones:
            # Find the corresponding file for this zone
            file_path = next(path for path in data_paths if zone in path)
            data, features, target, _ = load_data(file_path)
            dataset_obj = PharmacyDataset(data, features, target)
            dataloader = DataLoader(dataset_obj, batch_size=32, shuffle=False)
            
            # Get predictions from the model for this zone
            zone_model = pharmacy_models[data_paths.index(file_path)]
            predictions = predict_model(zone_model, dataloader)
            
            # Append predictions to city data
            data['Predicted_Cost'] = predictions
            data['Predicted_Medication_Name'] = data['Medication_Name']  # Ensure this matches the actual medication
            data['Predicted_Trend_Change'] = np.random.randn(len(predictions))  # Example trend changes
            city_data.append(data[['Medication_Name', 'Predicted_Cost', 'Predicted_Trend_Change']])
        
        # Aggregate city data
        aggregated_city_data = pd.concat(city_data).drop_duplicates()
        
        # Just before saving, average the values for each Medication_Name
        aggregated_city_data = aggregated_city_data.groupby('Medication_Name').agg({
            'Predicted_Cost': 'mean',
            'Predicted_Trend_Change': 'mean'
        }).reset_index()
        
        city_predictions[f"City{city_id}"] = aggregated_city_data
    
    # For each city, save the aggregated prediction after averaging
    for city, city_data in city_predictions.items():
        city_data.to_csv(os.path.join(output_path, f"{city}_predictions.csv"), index=False)
        
    return city_predictions

# Step 4: Aggregate city-level models into one national-level model
def aggregate_city_models(city_predictions, output_path):
    national_predictions = pd.concat([city_data for city_data in city_predictions.values()]).drop_duplicates()
    
    # Just before saving, average the values for each Medication_Name
    national_predictions = national_predictions.groupby('Medication_Name').agg({
        'Predicted_Cost': 'mean',
        'Predicted_Trend_Change': 'mean'
    }).reset_index()
    
    # Save national-level aggregated predictions after averaging
    national_predictions.to_csv(os.path.join(output_path, "national_aggregated_predictions.csv"), index=False)
    return national_predictions

# Main Execution
if __name__ == "__main__":
    data_path = "./"  # Path where generated datasets are stored
    output_path = "./output/"  # Path to save outputs
    os.makedirs(output_path, exist_ok=True)
    
    # Define the paths for pharmacy datasets (using format Ph{p:02d}_Z{z:02d}_C{c:02d})
    pharmacy_data_paths = [os.path.join(data_path, f"Ph{p:02d}_Z{z:02d}_C{c:02d}_train.csv") 
                           for p in range(1, 5) for z in range(1, 4) for c in range(1, 4)]
    
    # Step 1: Gather all unique medication names
    medication_names = gather_medication_names(pharmacy_data_paths)
    print("All medication names:", medication_names)
    
    # Step 2: Train models for all pharmacies
    pharmacy_models = train_pharmacy_models(pharmacy_data_paths, output_path)
    
    # Step 3: Aggregate predictions for city level
    city_predictions = aggregate_pharmacy_models(pharmacy_models, pharmacy_data_paths, output_path)
    
    # Step 4: Aggregate predictions for national level
    national_predictions = aggregate_city_models(city_predictions,output_path)
    
    # Save the results for each city and the national level
    for city, city_data in city_predictions.items():
        city_data.to_csv(os.path.join(output_path, f"{city}_predictions.csv"), index=False)
    
    national_predictions.to_csv(os.path.join(output_path, "national_aggregated_predictions.csv"), index=False)

    print("City and National level aggregation complete.")


All medication names: ['Medication_28', 'Medication_12', 'Medication_9', 'Medication_11', 'Medication_4', 'Medication_16', 'Medication_13', 'Medication_18', 'Medication_29', 'Medication_8', 'Medication_24', 'Medication_3', 'Medication_6', 'Medication_22', 'Medication_23', 'Medication_30', 'Medication_27', 'Medication_26', 'Medication_25', 'Medication_17', 'Medication_21', 'Medication_1', 'Medication_7', 'Medication_14', 'Medication_10', 'Medication_5', 'Medication_20', 'Medication_15', 'Medication_19', 'Medication_2']
Training model for ./Ph01_Z01_C01_train.csv...
Epoch 1/2, Loss: 0.0636
Epoch 2/2, Loss: 0.0323
Model saved to ./output/Ph01_Z01_C01_train_model.pth
Training model for ./Ph01_Z01_C02_train.csv...
Epoch 1/2, Loss: 0.1088
Epoch 2/2, Loss: 0.0318
Model saved to ./output/Ph01_Z01_C02_train_model.pth
Training model for ./Ph01_Z01_C03_train.csv...
Epoch 1/2, Loss: 0.0556
Epoch 2/2, Loss: 0.0300
Model saved to ./output/Ph01_Z01_C03_train_model.pth
Training model for ./Ph01_Z02_C0

C:\Users\Nanotec Ss\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/2, Loss: 0.0692


C:\Users\Nanotec Ss\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 2/2, Loss: 0.0288
Model saved to ./output/Ph02_Z01_C03_train_model.pth
Training model for ./Ph02_Z02_C01_train.csv...
Epoch 1/2, Loss: 0.0657
Epoch 2/2, Loss: 0.0326
Model saved to ./output/Ph02_Z02_C01_train_model.pth
Training model for ./Ph02_Z02_C02_train.csv...
Epoch 1/2, Loss: 0.0679
Epoch 2/2, Loss: 0.0326
Model saved to ./output/Ph02_Z02_C02_train_model.pth
Training model for ./Ph02_Z02_C03_train.csv...
Epoch 1/2, Loss: 0.0580
Epoch 2/2, Loss: 0.0315
Model saved to ./output/Ph02_Z02_C03_train_model.pth
Training model for ./Ph02_Z03_C01_train.csv...
Epoch 1/2, Loss: 0.0613
Epoch 2/2, Loss: 0.0316
Model saved to ./output/Ph02_Z03_C01_train_model.pth
Training model for ./Ph02_Z03_C02_train.csv...
Epoch 1/2, Loss: 0.0754
Epoch 2/2, Loss: 0.0315
Model saved to ./output/Ph02_Z03_C02_train_model.pth
Training model for ./Ph02_Z03_C03_train.csv...
Epoch 1/2, Loss: 0.0624
Epoch 2/2, Loss: 0.0304
Model saved to ./output/Ph02_Z03_C03_train_model.pth
Training model for ./Ph03_Z01_C01_t

C:\Users\Nanotec Ss\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/2, Loss: 0.0542


C:\Users\Nanotec Ss\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 2/2, Loss: 0.0294
Model saved to ./output/Ph04_Z01_C01_train_model.pth
Training model for ./Ph04_Z01_C02_train.csv...
Epoch 1/2, Loss: 0.0906
Epoch 2/2, Loss: 0.0335
Model saved to ./output/Ph04_Z01_C02_train_model.pth
Training model for ./Ph04_Z01_C03_train.csv...
Epoch 1/2, Loss: 0.0588
Epoch 2/2, Loss: 0.0269
Model saved to ./output/Ph04_Z01_C03_train_model.pth
Training model for ./Ph04_Z02_C01_train.csv...
Epoch 1/2, Loss: 0.1159
Epoch 2/2, Loss: 0.0336
Model saved to ./output/Ph04_Z02_C01_train_model.pth
Training model for ./Ph04_Z02_C02_train.csv...
Epoch 1/2, Loss: 0.0998
Epoch 2/2, Loss: 0.0298
Model saved to ./output/Ph04_Z02_C02_train_model.pth
Training model for ./Ph04_Z02_C03_train.csv...
Epoch 1/2, Loss: 0.0722
Epoch 2/2, Loss: 0.0353
Model saved to ./output/Ph04_Z02_C03_train_model.pth
Training model for ./Ph04_Z03_C01_train.csv...
Epoch 1/2, Loss: 0.0586
Epoch 2/2, Loss: 0.0288
Model saved to ./output/Ph04_Z03_C01_train_model.pth
Training model for ./Ph04_Z03_C02_t